In [1]:
import pandas as pd

df_idx = pd.read_csv('../data/processed/avg_ratings.csv')
df_corpus = pd.read_table('../data/raw/full_lyrics.tsv')

In [2]:
df_idx

,pair_id,sim_rating,song_1,song_2,length_1,length_2
0,10052_10842,1.333333,10052,10842,115,31
1,10052_11142,1.000000,10052,11142,115,105
2,10052_21856,1.000000,10052,21856,115,78
3,10052_24450,1.666667,10052,24450,115,108
4,10052_25047,0.000000,10052,25047,115,49
...,...,...,...,...,...,...
671,8863_55236,1.333333,8863,55236,64,61
672,9975_22822,2.333333,9975,22822,139,78
673,9975_24450,2.666667,9975,24450,139,108
674,9975_34125,0.000000,9975,34125,139,17


In [3]:
df_corpus

,song_id,text,artist_name,track_name
0,5439,"Dulce brillo\nque aún estaba afuera,\ncurtiend...",Eduardo Mateo,Dulce Brillo
1,29094,"No exactamente yo, aunque sí,\npero no lo soy....",Maga,"Sí, Pero No Lo Soy"
2,69790,"Mi novia Ximenita\nle gusta pelear,\npatea de ...",Los Piojos,Ximenita
3,66730,Oh de profundis.\nOh de procella.\nAh Canto.\n...,Arcana,Chant of the Awakening
4,28437,"Hasta la punta de tu luz,\nhasta tu grito más ...",Arbolito,La Máquina
...,...,...,...,...
70,38000,Me gusta cantarle al viento\nporque vuelan mis...,El Halcon De La Sierra,La Feria De Las Flores
71,48594,Y en la plaza de mi pueblo\ndijo el jornalero ...,Reincidentes,Jornaleros Andaluces
72,37194,Me voy a tomar\nese vino que pensás dejar\ny n...,Bulldog,Un Vinito Mas
73,25047,PI PA PU\nPA PA PI PI PA PIUI\nPI PA PI\nPA PI...,Los Visitantes,Pi-Pa-Pu


In [4]:
_df = df_corpus.set_index('song_id')
df = pd.DataFrame()
df['text1']=_df.loc[df_idx.song_1].text.reset_index(drop=['song_id'])
df['text2']=_df.loc[df_idx.song_2].text.reset_index(drop=['song_id'])
df.reset_index(inplace=True, drop=['index'])
df['id1'] = df_idx.song_1
df['id2'] = df_idx.song_2
df['labels'] = df_idx.sim_rating


In [5]:
df

,text1,text2,id1,id2,labels
0,"Adiós, adiós, mi México lindo\nmañana, mañana ...",Bueno... Nunca me podré bajar\ndel planeta cer...,10052,10842,1.333333
1,"Adiós, adiós, mi México lindo\nmañana, mañana ...",Yo tengo para ti\nun sitio especial en mi nuev...,10052,11142,1.000000
2,"Adiós, adiós, mi México lindo\nmañana, mañana ...","Soy famoso en todo el mundo,\npor mi arte y mi...",10052,21856,1.000000
3,"Adiós, adiós, mi México lindo\nmañana, mañana ...",Volviendo el rostro hacia atrás\nella midió su...,10052,24450,1.666667
4,"Adiós, adiós, mi México lindo\nmañana, mañana ...",PI PA PU\nPA PA PI PI PA PIUI\nPI PA PI\nPA PI...,10052,25047,0.000000
...,...,...,...,...,...
671,¿Qué le voy a decir si se va?\n¿Que vuelva?\n¿...,Este es el camino\ndonde todos buscan algo.\nY...,8863,55236,1.333333
672,Cuantas noches fatídicas de vicio\nTus ilusion...,Mirando la hiedra\ntu recuerdo me envenena.\nT...,9975,22822,2.333333
673,Cuantas noches fatídicas de vicio\nTus ilusion...,Volviendo el rostro hacia atrás\nella midió su...,9975,24450,2.666667
674,Cuantas noches fatídicas de vicio\nTus ilusion...,"Señor, ten piedad de nosotros.\nTen piedad.\nS...",9975,34125,0.000000


Counter({66730: 99, 37485: 69, 28437: 63, 5439: 63, 55093: 60, 29094: 57, 69790: 57, 37461: 57, 35331: 57, 28616: 51, 37416: 48, 48054: 45, 14251: 42, 16987: 42, 3348: 42, 46366: 42, 31841: 39, 62264: 39, 29590: 36, 34119: 36, 22840: 36, 8851: 33, 37844: 33, 30582: 33, 235: 30, 33088: 30, 31722: 30, 36132: 30, 8842: 30, 24369: 30, 26165: 30, 27790: 27, 1868: 27, 32204: 27, 25639: 27, 34125: 24, 3071: 24, 33902: 24, 27446: 24, 16872: 24, 76075: 24, 21227: 24, 37741: 21, 59899: 21, 5088: 21, 15357: 21, 21856: 18, 10052: 18, 10842: 18, 76479: 18, 7291: 18, 24450: 15, 22822: 15, 7727: 15, 72281: 15, 6632: 15, 19354: 12, 52438: 12, 11068: 12, 9975: 12, 12971: 9, 14975: 9, 8719: 9, 11142: 6, 38000: 6, 8863: 6, 69868: 6, 25047: 3, 48594: 3, 12476: 3, 19929: 3, 37194: 3})
